In [1]:
import pandas as pd
import numpy as np
import sklearn

In [2]:
df = pd.read_csv("Qualitative_Bankruptcy_data.txt")

Attribute Information: (P=Positive,A-Average,N-negative,B-Bankruptcy,NB-Non-Bankruptcy)
Industrial Risk: {P,A,N}
Management Risk: {P,A,N}
Financial Flexibility: {P,A,N}
Credibility: {P,A,N}
Competitiveness: {P,A,N}
Operating Risk: {P,A,N}
Class: {B,NB} 

In [5]:
df.head()

,P,P.1,A,A.1,A.2,P.2,NB
0,N,N,A,A,A,N,NB
1,A,A,A,A,A,A,NB
2,P,P,P,P,P,P,NB
3,N,N,P,P,P,N,NB
4,A,A,P,P,P,A,NB


In [6]:
from sklearn.preprocessing import LabelEncoder

In [7]:
le_ir = LabelEncoder()
le_mr = LabelEncoder()
le_ff = LabelEncoder()
le_cr = LabelEncoder()
le_co = LabelEncoder()
le_or = LabelEncoder()
le_cl = LabelEncoder()

In [8]:
df['ir_n'] = le_ir.fit_transform(df.P)
df['mr_n'] = le_ir.fit_transform(df["P.1"])
df['ff_n'] = le_ir.fit_transform(df.A)
df['cr_n'] = le_ir.fit_transform(df["A.1"])
df['co_n'] = le_ir.fit_transform(df['A.2'])
df['or_n'] = le_ir.fit_transform(df["P.2"])
df['cl_n'] = le_ir.fit_transform(df.NB)

In [9]:
df

,P,P.1,A,A.1,A.2,P.2,NB,ir_n,mr_n,ff_n,cr_n,co_n,or_n,cl_n
0,N,N,A,A,A,N,NB,1,1,0,0,0,1,1
1,A,A,A,A,A,A,NB,0,0,0,0,0,0,1
2,P,P,P,P,P,P,NB,2,2,2,2,2,2,1
3,N,N,P,P,P,N,NB,1,1,2,2,2,1,1
4,A,A,P,P,P,A,NB,0,0,2,2,2,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
244,N,N,A,N,N,N,B,1,1,0,1,1,1,0
245,P,N,N,N,N,N,B,2,1,1,1,1,1,0
246,A,N,N,N,N,N,B,0,1,1,1,1,1,0
247,N,N,N,N,N,N,B,1,1,1,1,1,1,0


In [14]:
inputs = df[['ir_n', 'mr_n', 'ff_n', 'cr_n', 'co_n', 'or_n']]

In [15]:
inputs

,ir_n,mr_n,ff_n,cr_n,co_n,or_n
0,1,1,0,0,0,1
1,0,0,0,0,0,0
2,2,2,2,2,2,2
3,1,1,2,2,2,1
4,0,0,2,2,2,0
...,...,...,...,...,...,...
244,1,1,0,1,1,1
245,2,1,1,1,1,1
246,0,1,1,1,1,1
247,1,1,1,1,1,1


In [16]:
output = df[['cl_n']]

In [17]:
output

,cl_n
0,1
1,1
2,1
3,1
4,1
...,...
244,0
245,0
246,0
247,0


In [25]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(inputs, output, test_size=0.33, random_state=42)

In [26]:
from sklearn.tree import DecisionTreeClassifier
Dtree = DecisionTreeClassifier()

In [27]:
Dtree.fit(X_train, y_train)

DecisionTreeClassifier()

In [28]:
Dtree.score(X_train, y_train)

1.0

In [30]:
Dtree.score(X_test, y_test)

1.0

In [ ]:
#Seems a bit suspicious to me that they're the same number. 
#I'm using categorical cross entropy since this is a classification problem

In [32]:
import matplotlib.pyplot as plt

In [45]:
#1. Saving the image of the decision as a png   
plt.subplots(figsize=(17, 12))
sklearn.tree.plot_tree(Dtree, feature_names=inputs.columns, filled=True, rounded=True, class_names=["B", "NB"])
plt.savefig("decision_tree.png")
# 2. Output the tree as text in the console
tree_as_text = sklearn.tree.export_text(Dtree, feature_names=list(inputs.columns))
print(tree_as_text)

|--- co_n <= 1.50
|   |--- co_n <= 0.50
|   |   |--- ir_n <= 1.50
|   |   |   |--- class: 1
|   |   |--- ir_n >  1.50
|   |   |   |--- or_n <= 0.50
|   |   |   |   |--- class: 0
|   |   |   |--- or_n >  0.50
|   |   |   |   |--- class: 1
|   |--- co_n >  0.50
|   |   |--- class: 0
|--- co_n >  1.50
|   |--- class: 1



In [46]:
from sklearn.ensemble import BaggingClassifier

In [47]:
bagging = BaggingClassifier()

In [50]:
np.reshape(y_train, (166,)).shape

(166, 1)

In [48]:
bagging.fit(X_train, y_train)

/Users/Andrew/miniforge3/envs/tf_m1/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


BaggingClassifier()

In [51]:
bagging.score(X_train,y_train)

1.0

In [52]:
bagging.score(X_test, y_test)

1.0

In [53]:
from sklearn.ensemble import GradientBoostingClassifier

In [54]:
boosting = GradientBoostingClassifier()

In [55]:
boosting.fit(X_train, y_train)

/Users/Andrew/miniforge3/envs/tf_m1/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


GradientBoostingClassifier()

In [56]:
boosting.score(X_train, y_train)

1.0

In [57]:
boosting.score(X_test, y_test)

0.9879518072289156

I think the model has overfit on almost all cases but im happy that i get some error in the test environment